In [1]:
import os

In [2]:
pwd

'd:\\cnn-example\\cnn-chickendisease-example\\research'

In [3]:
os.chdir('../')

In [4]:
pwd

'd:\\cnn-example\\cnn-chickendisease-example'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    "data ingestion class"
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        "download data from github URL"
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# updating the pipeline step
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

# inside the artifacts you can see it has created data ingestion folder
# inside that it has downloaded the data and extracted the zip file

# now convert this notebook into modular piece
# copy paste the entity created in config_entity.py in entity folder

[2025-01-02 09:23:04,192: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-02 09:23:04,196: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 09:23:04,200: INFO: common: created directory at: artifacts]
[2025-01-02 09:23:04,202: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-02 09:23:06,623: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 1550292
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5f0daeff07d0400128ba80794d989bce7acad6d32d6611ad3fa2a1a1408fc961"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: CFEC:1F7047:7BB609:B8B06D:677616A5
Accept-Ranges: bytes
Date: Thu, 02 Jan 2025 04:31:33 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100032-HYD
X-Cache: